In [941]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "06" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 19
repetitionName = "udrlBC4CVTest"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

40
4.444444444444445


In [942]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 147)
(10, 147)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.745,0.735,0.73,0.71,0.7,0.705,0.745,0.74,0.725,0.7,0.71,0.71,0.735,0.74,0.725,0.71,0.705,0.71,0.76,0.74,0.73,0.72,0.71,0.715,0.75,0.735,0.715,0.705,0.71,0.705,0.75,0.735,0.73,0.72,0.7,0.715,0.765,0.735,0.73,0.7,0.7,0.705,0.74,0.75,0.73,0.7,0.71,0.7,0.725,0.725,0.71,0.705,0.685,0.7,0.73,0.735,0.71,0.715,0.705,0.715,0.755,0.735,0.72,0.7,0.71,0.71,0.745,0.735,0.73,0.72,0.705,0.71,0.755,0.745,0.725,0.72,0.715,0.71,0.75,0.755,0.71,0.715,0.705,0.7,0.745,0.735,0.735,0.715,0.715,0.705,0.745,0.735,0.73,0.695,0.71,0.705,0.755,0.74,0.73,0.695,0.71,0.695,0.745,0.745,0.695,0.715,0.725,0.715,0.745,0.725,0.715,0.69,0.705,0.71,0.755,0.74,0.73,0.705,0.7,0.705,0.75,0.74,0.73,0.71,0.7,0.705,0.71,0.73,0.7,0.685,0.71,0.7,0.67,0.68,0.68,0.65,0.68,0.695,0.75,0.73,0.715,0.7,0.71,0.71,0.71,0.73,0.705
2,0.704167,0.754167,0.704167,0.758333,0.75,0.729167,0.704167,0.75,0.704167,0.754167,0.75,0.729167,0.691667,0.75,0.7,0.758333,0.75,0.733333,0.704167,0.741667,0.708333,0.758333,0.75,0.729167,0.695833,0.741667,0.7,0.741667,0.741667,0.716667,0.708333,0.754167,0.704167,0.75,0.75,0.729167,0.708333,0.754167,0.704167,0.754167,0.75,0.729167,0.7,0.754167,0.704167,0.758333,0.758333,0.733333,0.7125,0.754167,0.704167,0.775,0.7625,0.733333,0.6875,0.733333,0.708333,0.7375,0.725,0.725,0.7,0.75,0.7,0.754167,0.75,0.725,0.7125,0.754167,0.704167,0.75,0.75,0.729167,0.7125,0.754167,0.704167,0.745833,0.75,0.720833,0.6875,0.741667,0.691667,0.7375,0.7375,0.704167,0.7,0.758333,0.7,0.741667,0.741667,0.720833,0.720833,0.745833,0.708333,0.758333,0.75,0.733333,0.7125,0.745833,0.708333,0.7625,0.754167,0.729167,0.691667,0.745833,0.7,0.741667,0.741667,0.704167,0.6875,0.741667,0.708333,0.733333,0.7375,0.7125,0.708333,0.754167,0.704167,0.75,0.75,0.729167,0.708333,0.75,0.704167,0.754167,0.75,0.729167,0.708333,0.733333,0.704167,0.745833,0.729167,0.725,0.679167,0.708333,0.679167,0.704167,0.716667,0.679167,0.6875,0.75,0.7,0.75,0.75,0.725,0.741667,0.754167,0.7125
3,0.627778,0.633333,0.677778,0.661111,0.683333,0.677778,0.65,0.627778,0.677778,0.65,0.688889,0.677778,0.644444,0.622222,0.672222,0.661111,0.683333,0.672222,0.638889,0.616667,0.677778,0.644444,0.666667,0.683333,0.638889,0.622222,0.677778,0.644444,0.672222,0.683333,0.638889,0.633333,0.677778,0.655556,0.683333,0.677778,0.638889,0.633333,0.677778,0.661111,0.683333,0.677778,0.638889,0.633333,0.672222,0.655556,0.672222,0.677778,0.638889,0.638889,0.677778,0.666667,0.7,0.694444,0.638889,0.65,0.672222,0.638889,0.638889,0.661111,0.633333,0.638889,0.672222,0.65,0.666667,0.683333,0.65,0.633333,0.677778,0.661111,0.683333,0.677778,0.65,0.633333,0.683333,0.644444,0.672222,0.677778,0.65,0.644444,0.672222,0.644444,0.677778,0.683333,0.65,0.616667,0.677778,0.661111,0.661111,0.677778,0.622222,0.622222,0.672222,0.638889,0.683333,0.683333,0.644444,0.622222,0.67

In [943]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.79 : subject 5, feature:dataGCV-BCcn3-fftDatacn3BC-dataGCV-BC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 4, feature:gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataCorr1d, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 4, feature:gaussianDatacn3BC-fftDatacn3BC-dataCorr1dcn3--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 4, feature:gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataHR, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianDatacn3BC-fftDatacn3BC-welchDataBC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataHR, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataCorr1d, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianDatacn3BC-fftDatacn3BC-dataCorr1dcn3--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 4, feature:gaussianDatacn3BC-fftDatacn3BC-fftDatacn3--fftDataBC, kernelsig

In [944]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.79 : subject 5, feature:dataGCV-BCcn3-fftDatacn3BC-dataGCV-BC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataHR, kernelsigmoid, C:2.5

Accuracy 0.79 : subject 4, feature:gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataCorr1d, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 4, feature:gaussianDatacn3BC-fftDatacn3BC-dataCorr1dcn3--fftDataBC, kernelsigmoid, C:2.5

Accuracy 0.78 : subject 2, feature:dataGCV-BCcn3-fftDatacn3BC-welchDataBC--fftDataBC, kernellinear, C:0.5
Accuracy 0.76 : subject 2, feature:dataGCV-BCcn3-fftDatacn3BC-welchDatacn3--fftDataBC, kernellinear, C:0.5

Accuracy 0.77 : subject 9, feature:dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataHRCV-BC, kernellinear, C:0.5
Accuracy 0.77 : subject 9, feature:dataGCV-BCcn3-fftDatacn3BC-fftDataBC--welchData, kernellinear, C:0.5

Accuracy 0.77 : subject 8, feature:gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5

In [945]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.71887755 0.72820295 0.65854119 0.75901361 0.74702381
 0.68738404 0.71407313 0.73337585 0.72693452]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146
0,0.71401,0.725974,0.717815,0.723421,0.721233,0.719835,0.719952,0.724987,0.717144,0.720213,0.724235,0.720275,0.714162,0.725643,0.716758,0.726651,0.723062,0.718269,0.722983,0.727172,0.718636,0.726437,0.724017,0.721806,0.711918,0.728159,0.715464,0.720319,0.730653,0.719421,0.721808,0.727132,0.716552,0.723915,0.721233,0.720946,0.719487,0.725974,0.717815,0.72433,0.721812,0.719835,0.713932,0.724874,0.714546,0.725217,0.723488,0.718248,0.718273,0.729174,0.718255,0.728512,0.730299,0.716449,0.710979,0.723554,0.710356,0.708208,0.713326,0.708815,0.715497,0.726718,0.716087,0.720728,0.726511,0.719104,0.72093,0.726553,0.717815,0.726784,0.722367,0.720969,0.719611,0.725928,0.715109,0.717598,0.725885,0.717687,0.71512,0.723665,0.71487,0.715759,0.722865,0.714005,0.717521,0.721187,0.717571,0.718484,0.723619,0.714122,0.715245,0.72434,0.718071,0.719401,0.724943,0.720568,0.716626,0.724601,0.716619,0.720225,0.722846,0.715136,0.718321,0.725812,0.714115,0.721339,0.722752,0.717302,0.715041,0.728206,0.715233,0.718035,0.729133,0.719514,0.714672,0.725488,0.716514,0.722764,0.721195,0.719088,0.71795,0.726067,0.717815,0.720682,0.722391,0.719835,0.715424,0.722925,0.709298,0.725894,0.721885,0.713489,0.693885,0.695617,0.698455,0.692116,0.696207,0.700507,0.714555,0.726162,0.716727,0.723082,0.726511,0.719142,0.717736,0.720887,0.715612


Max average accuracy 0.7306530583613918


In [946]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataCorr1d
sigmoid
0.7306530583613918

dataGCV-BCcn3-fftDatacn3BC-welchDataBC--fftDataBC
sigmoid
0.7302988215488215

gaussianDatacn3BC-fftDatacn3BC-welchDataBC--fftDataBC
sigmoid
0.7291736812570146

dataGCV-BCcn3-fftDatacn3BC-dataCorr1dcn3--fftDataBC
sigmoid
0.7291329966329968

dataGCV-BCcn3-fftDatacn3BC-welchDataBC--fftDataBC
linear
0.7285122053872054



In [947]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

gaussianDatacn3BC-fftDatacn3BC--fftDataBC
1
dataGCV-BCcn3-fftDatacn3BC--fftDataBC
2
gaussianDatacn3BC-fftDatacn3BC-fftDataBC--fftData
3
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--fftData
4
gaussianDatacn3BC-fftDatacn3BC-fftDataBC--welchData
5
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--welchData
6
gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataHR
7
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataHR
8
gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataCorr1d
9
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataCorr1d
10
gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataFFTCV-BC
11
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataFFTCV-BC
12
gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataWCV-BC
13
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataWCV-BC
14
gaussianDatacn3BC-fftDatacn3BC-fftDataBC--dataHRCV-BC
15
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataHRCV-BC
16
gaussianDatacn3BC-fftDatacn3BC-welchDataBC--fftDataBC
17
dataGCV-BCcn3-fftDatacn3BC-welchDataBC--fftDataBC
18
gaussianDatacn3BC-fftDatacn3BC-dataHRBC--fftDataBC
19
dataGCV-BCcn3

In [948]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146
0,6.426092,6.533769,6.460335,6.510789,6.491098,6.478516,6.479564,6.52488,6.454293,6.481913,6.518112,6.482475,6.427456,6.530783,6.450821,6.539861,6.507557,6.464419,6.506843,6.544545,6.467721,6.537929,6.516155,6.49625,6.40726,6.553434,6.439179,6.482872,6.575878,6.474792,6.496275,6.544186,6.448971,6.515234,6.491098,6.488516,6.475385,6.533769,6.460335,6.518971,6.496307,6.478516,6.425385,6.523864,6.430915,6.526957,6.511389,6.464236,6.464457,6.562563,6.464293,6.55661,6.572689,6.448043,6.398813,6.511989,6.393207,6.37387,6.419931,6.379331,6.43947,6.540461,6.444779,6.486553,6.538598,6.471932,6.488371,6.538977,6.460335,6.541054,6.501307,6.488725,6.476496,6.533352,6.435985,6.458384,6.532961,6.459179,6.43608,6.512986,6.433832,6.441831,6.505789,6.426042,6.457689,6.490682,6.458138,6.466357,6.512569,6.427096,6.437203,6.519059,6.462639,6.474609,6.524489,6.485114,6.449634,6.521408,6.449571,6.482027,6.505612,6.436225,6.464886,6.532311,6.427039,6.492052,6.504766,6.45572,6.435372,6.553851,6.437096,6.462317,6.562197,6.475625,6.432052,6.529394,6.448624,6.504874,6.490751,6.471793,6.461553,6.534602,6.460335,6.486136,6.501515,6.478516,6.438819,6.506326,6.383681,6.533043,6.496963,6.421402,6.244968,6.260556,6.286092,6.22904,6.265859,6.304564,6.430997,6.535461,6.450543,6.507734,6.538598,6.472279,6.459621,6.487986,6.440511


28
9.0
Max total significant accuracy then averaged 0.7306530583613918 
dataGCV-BCcn3-fftDatacn3BC-fftDataBC--dataCorr1d
sigmoid
C: 2.5 


In [949]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
